## 1. 분류 모형 활용 사례

## 2. Smote + MLP

In [ ]:
!pip install pycaret

In [2]:
!unzip caravan.zip

Archive:  caravan.zip
  inflating: caravan.csv             


In [17]:
import pandas as pd
data = pd.read_csv("caravan.csv")
print( data.Purchase.value_counts() )

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X  = data.drop("Purchase", axis=1)
X_scaled = StandardScaler().fit_transform( X )
y = data.Purchase

X_train, X_test,y_train,y_test = train_test_split(X_scaled,y,test_size=0.2, stratify=y)

0    5474
1     348
Name: Purchase, dtype: int64


In [18]:
from imblearn.over_sampling import SMOTE
X_train_res, y_train_res = SMOTE(sampling_strategy = 0.25).fit_resample( X_train, y_train )

In [19]:
from sklearn.neural_network import MLPClassifier
model = MLPClassifier( hidden_layer_sizes=(24,12 ) )
model.fit( X_train_res, y_train_res)

MLPClassifier(hidden_layer_sizes=(24, 12))

In [20]:
from sklearn.metrics import classification_report
print( classification_report( y_test, model.predict( X_test)) )

              precision    recall  f1-score   support

           0       0.94      0.92      0.93      1095
           1       0.10      0.13      0.11        70

    accuracy                           0.88      1165
   macro avg       0.52      0.53      0.52      1165
weighted avg       0.89      0.88      0.88      1165



- pycaret 비교

In [7]:
from pycaret.classification import *

In [22]:
tmp = pd.DataFrame(X_train_res)
tmp["Purchase"] = y_train_res

In [23]:
clf = setup(data=tmp, target='Purchase', train_size=0.8, session_id=10, normalize=True)

,Description,Value
0,Session id,10
1,Target,Purchase
2,Target type,Binary
3,Original data shape,"(5473, 86)"
4,Transformed data shape,"(5473, 86)"
5,Transformed train set shape,"(4378, 86)"
6,Transformed test set shape,"(1095, 86)"
7,Numeric features,85
8,Preprocess,True
9,Imputation type,simple


In [24]:
models = compare_models(sort='Recall', n_select=3, fold=2)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
nb,Naive Bayes,0.2442,0.5489,0.9829,0.2070,0.3420,0.0177,0.0764,0.1100
xgboost,Extreme Gradient Boosting,0.9246,0.9341,0.7360,0.8668,0.7961,0.7502,0.7540,0.7850
knn,K Neighbors Classifier,0.7990,0.8616,0.7212,0.4992,0.5895,0.4624,0.4764,0.2600
lightgbm,Light Gradient Boosting Machine,0.9237,0.9365,0.7211,0.8752,0.7907,0.7446,0.7498,2.5550
dt,Decision Tree Classifier,0.8696,0.8130,0.7063,0.6631,0.6840,0.6019,0.6024,0.1400
gbc,Gradient Boosting Classifier,0.9287,0.9348,0.6731,0.9576,0.7905,0.7492,0.7661,1.2750
et,Extra Trees Classifier,0.9095,0.9280,0.6708,0.8445,0.7476,0.6934,0.7003,0.8650
rf,Random Forest Classifier,0.9109,0.9338,0.6411,0.8807,0.7421,0.6898,0.7025,0.5400
ada,Ada Boost Classifier,0.8954,0.8905,0.6263,0.8069,0.7052,0.6428,0.6505,0.3600
svm,SVM - Linear Kernel,0.7650,0.0000,0.4320,0.4154,0.4235,0.2760,0.2761,0.1250


Processing:   0%|          | 0/67 [00:00<?, ?it/s]

In [25]:
tmp_test = pd.DataFrame(X_test)

model1 = tune_model(models[2], optimize="Prec.")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8904,0.9293,0.6552,0.7600,0.7037,0.6369,0.6396
1,0.8904,0.9257,0.6667,0.7532,0.7073,0.6402,0.6420
2,0.8813,0.8849,0.6552,0.7215,0.6867,0.6137,0.6148
3,0.8607,0.8783,0.5795,0.6800,0.6258,0.5409,0.5435
4,0.9041,0.9319,0.7045,0.7949,0.7470,0.6881,0.6900
5,0.9041,0.9341,0.6932,0.8026,0.7439,0.6853,0.6881
6,0.8699,0.8881,0.6364,0.6914,0.6627,0.5823,0.5830
7,0.8676,0.9022,0.5795,0.7083,0.6375,0.5575,0.5617
8,0.8787,0.8940,0.5862,0.7500,0.6581,0.5857,0.5923


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [26]:
predicted =  predict_model(model1, tmp_test)
print(classification_report( y_test, predicted.prediction_label))

              precision    recall  f1-score   support

           0       0.94      0.95      0.95      1095
           1       0.14      0.13      0.13        70

    accuracy                           0.90      1165
   macro avg       0.54      0.54      0.54      1165
weighted avg       0.90      0.90      0.90      1165



## 3. MLP + GridSearch

In [27]:
from sklearn import model_selection
from sklearn import metrics

param_grid = {
    "hidden_layer_sizes":[(24,12),(12,6)]
}
#declaring GridSearchCV model
grid = model_selection.GridSearchCV(
    estimator = MLPClassifier(), #모형
    param_grid = param_grid,  #
    scoring = 'accuracy', #정분류율
    verbose = 10,
    n_jobs = 1,
    cv = 2      #k-fold CV, k값.
)
#fitting values to the gridsearchcv model
grid.fit(X_train_res, y_train_res)

#printing the best possible values to enhance accuracy
print(grid.best_params_)
print(grid.best_estimator_)
#printing the best score
print(grid.best_score_)



Fitting 2 folds for each of 2 candidates, totalling 4 fits
[CV 1/2; 1/2] START hidden_layer_sizes=(24, 12).................................
[CV 1/2; 1/2] END ..hidden_layer_sizes=(24, 12);, score=0.885 total time=   2.7s
[CV 2/2; 1/2] START hidden_layer_sizes=(24, 12).................................
[CV 2/2; 1/2] END ..hidden_layer_sizes=(24, 12);, score=0.918 total time=   2.7s
[CV 1/2; 2/2] START hidden_layer_sizes=(12, 6)..................................
[CV 1/2; 2/2] END ...hidden_layer_sizes=(12, 6);, score=0.874 total time=   1.6s
[CV 2/2; 2/2] START hidden_layer_sizes=(12, 6)..................................
[CV 2/2; 2/2] END ...hidden_layer_sizes=(12, 6);, score=0.872 total time=   1.9s
{'hidden_layer_sizes': (24, 12)}
MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(24, 12), learning_rate='constant',
              learning_rate_init=0.001, max_fun=

## 4. Keras Tuner 활용 하이퍼파라미터 튜닝

 - DNN의 하이터파라미터: 층의 수, 노드의 수, 학습율, 배치크기 등

In [28]:
import tensorflow as tf
from tensorflow import keras

In [29]:
!pip install -q -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 2.6 MB/s eta 0:00:00


In [30]:
import keras_tuner as kt

In [31]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [ ]:
img_train.shape
label_train.shape
img_train[0].reshape(784, )

In [38]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32').reshape(60000, 784) / 255.0
# 값을 변경(정수-> 실수), 모양을 2D로 변경, MinMaxScaling
img_test = img_test.astype('float32').reshape(10000, 784) / 255.0

In [ ]:
# 위의 자료에 대해서 lable_train을 범주를 하는 DNN을 구성하세요
# MLPClassifier : 48, 24
# tensorfolw : 48, 24
# target의 범주의 수는 10개 ->

# test에 대한 성능을 측정해보세요

model1 = MLPClassifier(hidden_)


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import


- 하이퍼튜닝을 위한 모형 구축 시 하이퍼 파라미터 탐색 공간 정의
- 하이퍼 모델: 하이퍼튜닝을 위해 설정하는 모델

- 하이퍼 모형의 설정
 - 모델 빌더 함수 사용: Keras Tuner API의 HyperModel 클래스를 하위 클래스화
 - 컴파일된 모델을 반환하고 인라인으로 정의한 하이퍼파라미터를 사용하여 모델을 하이퍼튜닝

In [ ]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(784, )))

  # Choose an optimal value between 32-96
  hp_units = hp.Int('units', min_value=32, max_value=96, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

- tuner 객체 만들고 튜닝 수행
 - randomsearch, hyperband, bayesianoptimization, sklearn 등의 tuner 가능
 - Hyperband 사용 시: objective와 epoch 지정

In [ ]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

In [ ]:
class ClearTrainingOutput(tf.keras.callbacks.Callback): #조기 학습 중단을 위해 사용
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [ ]:
#하이퍼 파라미터 탐색 실행
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[tf.keras.callbacks.EarlyStopping(patience=1)])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
optimal number of units:  {best_hps.get('units')}
optimal learning rate for the optimizer:  {best_hps.get('learning_rate')}
""")

Trial 8 Complete [00h 00m 16s]
val_accuracy: 0.09833333641290665

Best val_accuracy So Far: 0.12333333492279053
Total elapsed time: 00h 02m 13s

optimal number of units:  64
optimal learning rate for the optimizer:  0.001



In [ ]:
#탐색을 통해 찾은 하이퍼파라미터로 훈련 시 최적의 epoch을 발견
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=10, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/10
1500/1500 [==============================] - 9s 5ms/step - loss: 2.3025 - accuracy: 0.1044 - val_loss: 2.3016 - val_accuracy: 0.1027
Epoch 2/10
1500/1500 [==============================] - 7s 5ms/step - loss: 2.2999 - accuracy: 0.1110 - val_loss: 2.2968 - val_accuracy: 0.1075
Epoch 3/10
1500/1500 [==============================] - 6s 4ms/step - loss: 2.2900 - accuracy: 0.1480 - val_loss: 2.2806 - val_accuracy: 0.2557
Epoch 4/10
1500/1500 [==============================] - 6s 4ms/step - loss: 2.2669 - accuracy: 0.2247 - val_loss: 2.2493 - val_accuracy: 0.2792
Epoch 5/10
1500/1500 [==============================] - 6s 4ms/step - loss: 2.2280 - accuracy: 0.2694 - val_loss: 2.2027 - val_accuracy: 0.3434
Epoch 6/10
1500/1500 [==============================] - 7s 5ms/step - loss: 2.1764 - accuracy: 0.2890 - val_loss: 2.1470 - val_accuracy: 0.3134
Epoch 7/10
1500/1500 [==============================] - 7s 4ms/step - loss: 2.1185 - accuracy: 0.3222 - val_loss: 2.0873 - val_accuracy:

In [ ]:
#최적 모형 객체 만들기
hypermodel = tuner.hypermodel.build(best_hps)

# 최적 epoch으로 학습
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/10
1500/1500 [==============================] - 7s 5ms/step - loss: 2.3028 - accuracy: 0.0982 - val_loss: 2.3028 - val_accuracy: 0.0957
Epoch 2/10
1500/1500 [==============================] - 7s 4ms/step - loss: 2.3028 - accuracy: 0.0977 - val_loss: 2.3029 - val_accuracy: 0.0983
Epoch 3/10
1500/1500 [==============================] - 7s 4ms/step - loss: 2.3028 - accuracy: 0.0995 - val_loss: 2.3028 - val_accuracy: 0.0957
Epoch 4/10
1500/1500 [==============================] - 7s 5ms/step - loss: 2.3028 - accuracy: 0.0999 - val_loss: 2.3026 - val_accuracy: 0.1027
Epoch 5/10
1500/1500 [==============================] - 7s 5ms/step - loss: 2.3028 - accuracy: 0.0992 - val_loss: 2.3026 - val_accuracy: 0.0995
Epoch 6/10
1500/1500 [==============================] - 6s 4ms/step - loss: 2.3028 - accuracy: 0.0980 - val_loss: 2.3027 - val_accuracy: 0.0995
Epoch 7/10
1500/1500 [==============================] - 7s 4ms/step - loss: 2.3028 - accuracy: 0.0983 - val_loss: 2.3027 - val_accuracy:

In [ ]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 1s 2ms/step - loss: 2.2947 - accuracy: 0.2550
[test loss, test accuracy]: [2.294664144515991, 0.2549999952316284]


## 6. 유사도와 추천

In [39]:
import pandas as pd
df = pd.read_csv('movies.csv')

In [40]:
df.head()

,userId,title,rating,timestamp
0,1,American Pie,4.0,1260759139
1,4,American Pie,4.0,949896114
2,15,American Pie,4.0,1052896867
3,30,American Pie,2.0,994439964
4,34,American Pie,4.0,973747765


In [ ]:
df = df.pivot_table('rating', index = 'userId', columns = 'title')
df.head()

title,10 Things I Hate About You,12 Angry Men,1408,15 Minutes,16 Blocks,"20,000 Leagues Under the Sea",2001: A Space Odyssey,2046,21 Grams,25th Hour,...,Willy Wonka & the Chocolate Factory,World Trade Center,X-Men Origins: Wolverine,Y Tu Mamá También,You Only Live Twice,"You, Me and Dupree",Young Frankenstein,Zodiac,eXistenZ,xXx
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,5.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.shape

(670, 856)

- user들별로 평점을 주는 범위가 다르기 때문에 scaling하여 조정

In [ ]:
import numpy as np
df_scaled = df.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)

### Collaborative Filtering (협업 필터링) 리뷰
- 추천 시스템: 고객의 선호, 관심, 구매경력과 같은 개인화 정보를 기초로 고객에게 가장 알맞은 구매정보 제공
- 방식
  - Item-based collaborative filtering: item 간 similarity를 기반으로 추천
  - User-based collaborative filtering: user 간 similarity를 기반으로 추천
    - 성능이 다른 방식에 비해 떨어지는 것으로 알려져있음
  - Matrix factorization collaborative filtering: 잠재요인이 있다고 가정하여, 그 잠재요인을 행렬분해를 통해 찾아냄
    - 일반적으로 SVD(singular value decomposition)을 활용

- Input: user-item matrix (preference matrix)
  - 일반적으로 5점 척도로 구성된 rating + cosine similarity
  - binary인 경우 jaccard similarity 사용

- 한계
  - 단순 matrix를 사용하여 추천하기 때문에 context/content를 고려하지 않음
    - 최근에는 item의 text 정보, user의 context 정보들을 반영하여 딥러닝 기반의 추천 시스템 등장

### Item-based collaborative filtering
- Item*user으로 구성된 matrix를 사용하여 유사도 측정

In [ ]:
df_scaled = df_scaled.transpose().fillna(0)
df_scaled.head(5)

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-0.060526,0.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1408,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
15 Minutes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
16 Blocks,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


- user들 간 평점이 비슷한 정도를 기반으로 영화들의 유사도 평가

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
movie_sim = cosine_similarity(df_scaled, df_scaled)
print(movie_sim.shape)

(856, 856)


In [ ]:
movie_sim

In [ ]:
movie_sim = pd.DataFrame(movie_sim, index = df_scaled.index, columns = df_scaled.index)

In [ ]:
movie_sim['Snow White and the Seven Dwarfs'].sort_values(ascending=False)[:10]

title
Garfield                           1.000000
The Matrix                         1.000000
Snow White and the Seven Dwarfs    1.000000
Highlander                         1.000000
Alvin and the Chipmunks            0.922721
Prom Night                         0.913674
They Live                          0.558437
Zodiac                             0.546653
The Dark Knight                    0.491283
Shine a Light                      0.463028
Name: Snow White and the Seven Dwarfs, dtype: float64

## 7 .추천 라이브러리

In [41]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163751 sha256=5e19b966678b2c6f10cd58f0b341cd8ae507dc517f095d2c34071a11acac957d
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
df = pd.read_csv('movies.csv')
df

,userId,title,rating,timestamp
0,1,American Pie,4.0,1260759139
1,4,American Pie,4.0,949896114
2,15,American Pie,4.0,1052896867
3,30,American Pie,2.0,994439964
4,34,American Pie,4.0,973747765
...,...,...,...,...
18566,652,Lion of the Desert,5.0,1440269953
18567,652,Mud,5.0,1439586954
18568,659,K-PAX,4.0,836137550
18569,659,Starship Troopers,3.0,834694187


In [ ]:
from surprise import Reader, Dataset, KNNBasic, SVD
from surprise.model_selection import train_test_split, cross_validate
import pandas as pd

# Prepare the data to be used in Surprise
reader = Reader(rating_scale=(0,5))
data = Dataset.load_from_df(df[['userId', 'title', 'rating']], reader=reader)


In [ ]:
data

In [ ]:
sim_options = {
    'name': 'cosine',
    'user_based': True
}
algo = KNNBasic(sim_options=sim_options)

# Retrieve the trainset.
trainset, testset = train_test_split(data)
algo.fit(trainset)

# Predict
print(algo.predict(1, 5, r_ui=None, verbose=True))

Computing the cosine similarity matrix...
Done computing similarity matrix.
user: 1          item: 5          r_ui = None   est = 3.54   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1          item: 5          r_ui = None   est = 3.54   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}


In [ ]:
prediction = algo.test(testset)
prediction[:5]

[Prediction(uid=472, iid='Boogie Nights', r_ui=4.0, est=4.201173462840423, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=517, iid='Metropolis', r_ui=3.0, est=2.6268575319505496, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=23, iid='Being John Malkovich', r_ui=4.5, est=3.5271392246906523, details={'actual_k': 17, 'was_impossible': False}),
 Prediction(uid=523, iid='A Nightmare on Elm Street', r_ui=4.0, est=3.4626506634583705, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=105, iid='20,000 Leagues Under the Sea', r_ui=2.0, est=2.887541215716813, details={'actual_k': 40, 'was_impossible': False})]

- Surprise 지원 알고리즘
 - Random : 랜덤한 추천
 - Baseline : ALS(Alternating Least Square), SGD(Stochastic Gradient Descent)
 - Matrix factorization: SVD, SVD++
 - KNNs
 - Slope one, Co clustering

 - https://surprise.readthedocs.io/en/stable/prediction_algorithms_package.html

In [ ]:
from surprise import BaselineOnly
from surprise import KNNWithMeans, KNNBasic, KNNBaseline
from surprise import SVD
from surprise import SVDpp

from surprise import accuracy
from surprise import Dataset
from surprise.model_selection import train_test_split
import matplotlib.pyplot as plt


In [ ]:
trainset, testset = train_test_split(data, test_size = 0.25)

In [ ]:
recom = KNNBasic()  #CF
recom.fit(trainset)
predictions = recom.test(testset)
accuracy.rmse(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9505


0.9505290217572812

In [ ]:
recom = SVD()  #matrix 분해 (SVD) 이용
recom.fit(trainset)
predictions = recom.test(testset)
accuracy.rmse(predictions)

RMSE: 0.9034


0.9034162368979297

- ml-100k에 대해서 여러 추천 알고리즘을 수행하고 비교

In [ ]:
# MovieLens 100K
data = Dataset.load_builtin(name=u'ml-100k', prompt = False)
trainset, testset = train_test_split(data, test_size = 0.25)

Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [ ]:
algorithms = [KNNBasic, SVD]

algos=[]
rmses=[]

for i in algorithms:
  algo = i()
  algos.append(i.__name__)
  algo.fit(trainset)
  predictions = algo.test(testset)
  rmses.append(accuracy.rmse(predictions))

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9837
RMSE: 0.9371
